In [1]:
import re
def parse_summary(summary):
    bowler = "N/A"
    batsman = "N/A"
    runs = 0
    is_wicket = False
    bowler_score = 0  
    
    try:
        if " to " in summary:
            parts = summary.split(" to ", 1)
            bowler = parts[0].strip()
            rest = parts[1].strip()
            
            if "," in rest:
                subparts = rest.split(",", 1)
                batsman = subparts[0].strip()
                outcome = subparts[1].strip()
                
                if "OUT" in outcome or "WICKET" in outcome:
                    runs = 0
                    is_wicket = True
                    bowler_score = 5  # +5 for a wicket
                elif outcome == "no run":
                    runs = 0
                    bowler_score = 1  # +1 for no run
                elif "FOUR" in outcome or "4 run" in outcome:
                    runs = 4
                    bowler_score = -2  # -2 for a four
                elif "SIX" in outcome or "6 run" in outcome:
                    runs = 6
                    bowler_score = -3  # -3 for a six
                elif "leg bye" in outcome or "bye" in outcome:
                    if "leg bye" in outcome:
                        match = re.search(r'(\d+) leg bye', outcome)
                        runs = int(match.group(1)) if match else 1
                    else:  # regular bye
                        match = re.search(r'(\d+) bye', outcome)
                        runs = int(match.group(1)) if match else 1
                    bowler_score = -0.25  # -0.25 for leg bye or bye
                elif "wide" in outcome:
                    # Wides are always at least 1 run
                    match = re.search(r'(\d+) wide', outcome)
                    runs = int(match.group(1)) if match else 1
                    bowler_score = -1  # -1 for wide
                elif "no ball" in outcome:
                    # No balls are 1 run plus any runs scored
                    runs = 1  # Start with 1 for the no ball
                    
                    # Check if additional runs were scored
                    run_match = re.search(r'(\d+) run', outcome)
                    if run_match:
                        runs += int(run_match.group(1))
                    bowler_score = -1  # -1 for no ball (similar to wide)
                # Regular runs
                else:
                    match = re.search(r'(\d+) run', outcome)
                    if match:
                        runs = int(match.group(1))
                        # For regular runs (not 0, 4, or 6)
                        if runs > 0:
                            bowler_score = -runs / 2  # Negative score proportional to runs
        
        return bowler, batsman, runs, is_wicket, bowler_score
    except Exception as e:
        print(f"Error parsing summary: {e}")
        return "N/A", "N/A", 0, False, 0

In [2]:
def preprocess_raw_data(df):
    if df["Ball Number"].dtype == 'object':
        df[["over_str", "ball_str"]] = df["Ball Number"].str.split(".", expand=True)
        df["over"] = pd.to_numeric(df["over_str"], errors="coerce").fillna(-1).astype(int)
        df["ball"] = pd.to_numeric(df["ball_str"], errors="coerce").fillna(-1).astype(int)
        
        df["sort_order"] = df["over"] * 6 + df["ball"]
        df = df.sort_values(by=["sort_order"]).reset_index(drop=True)
        df = df.drop(columns=["over_str", "ball_str", "sort_order"])
    
    df["Ball"] = df.index + 1
    return df

In [3]:
def extract_commentary_data(page_source):
    soup = BeautifulSoup(page_source, "html.parser")
    
    ball_containers = soup.find_all("div", class_=lambda x: x and "ds-text-tight-l" in x and "ds-flex" in x)
    
    ball_numbers = []
    summaries = []
    descriptions = []
    
    for container in ball_containers:
        ball_span = container.find("span", class_="ds-text-tight-s ds-font-regular ds-mb-1 lg:ds-mb-0 lg:ds-mr-3 ds-block ds-text-center ds-text-typo-mid1")
        ball_text = ball_span.get_text(strip=True) if ball_span else "N/A"
        
        summary_div = container.find("div", class_="ds-leading-[16px] lg:ds-leading-none ds-mb-0.5")
        summary_text = summary_div.get_text(strip=True) if summary_div else "N/A"
        
        desc_p = container.find("p", class_="ci-html-content first-letter:ds-capitalize ds-leading-[24px]")
        desc_text = desc_p.get_text(strip=True) if desc_p else "N/A"
        
        ball_numbers.append(ball_text)
        summaries.append(summary_text)
        descriptions.append(desc_text)
    
    df = pd.DataFrame({
        "Ball Number": ball_numbers,
        "Summary": summaries,
        "Description": descriptions
    })
    
    return df

In [4]:
def analyze_match_data(df, match_id, target, city, stadium, player_names):
    # Function to find the full name from player_names set
    def get_full_name(short_name):
        # First check if the short name itself is in player_names
        if short_name in player_names:
            return short_name
        
        # Look for full names containing this short name
        for full_name in player_names:
            # Split full name into parts and check if any part matches the short name
            name_parts = full_name.split()
            if short_name in name_parts:
                return full_name
        
        # If no match found, return the original name
        return short_name
    
    # Apply the parse_summary function to extract data from the summary
    parsed_data = df["Summary"].apply(parse_summary)
    df['bowler'], df['batsman'], df['Runs'], df['is_wicket'], df['bowler_score'] = zip(*parsed_data)
    
    # Replace shortened names with full names from player_names set
    df['bowler'] = df['bowler'].apply(get_full_name)
    df['batsman'] = df['batsman'].apply(get_full_name)
    
    # Evaluate bowler performance for each ball
    df['bowler_good'] = df.apply(lambda row: 1 if row['bowler_score'] > 0 else 0, axis=1)
    
    # Ensure Runs column is numeric
    df['Runs'] = pd.to_numeric(df['Runs'], errors='coerce').fillna(0)
    
    # Add metadata
    df['match_id'] = match_id
    df['city'] = city
    df['stadium'] = stadium
    
    # Identify extras and calculate legal deliveries
    df['is_extra'] = df['Summary'].str.contains(r'wide|no ball', case=False, regex=True)
    df['legal_delivery'] = ~df['is_extra']
    df['legal_deliveries_count'] = df['legal_delivery'].cumsum()
    
    # Calculate overs properly based on legal deliveries
    df['overs_completed'] = (df['legal_deliveries_count'] - 1) // 6
    df['balls_in_current_over'] = ((df['legal_deliveries_count'] - 1) % 6) + 1
    
    # Handle the case where legal_deliveries_count is 0 (first ball)
    df.loc[df['legal_deliveries_count'] == 0, 'overs_completed'] = 0
    df.loc[df['legal_deliveries_count'] == 0, 'balls_in_current_over'] = 0
    
    # Calculate decimal overs
    df['overs_bowled'] = df['overs_completed'] + (df['balls_in_current_over'] / 6)
    df['overs_completed'] = df['overs_completed'].clip(lower=0)
    df['overs_bowled'] = df['overs_bowled'].clip(lower=0)
    
    # Calculate run statistics
    df['total_runs'] = df['Runs'].cumsum()
    df['current_run_rate'] = df.apply(
        lambda row: row['total_runs'] / max(row['overs_bowled'], 0.1) if row['overs_bowled'] > 0 else 0,
        axis=1
    )
    
    # Add match metadata and calculate target-related stats
    total_overs = 20  # Standard T20 match
    df['total_overs'] = total_overs
    df['target'] = target
    df['runs_needed'] = np.maximum(target - df['total_runs'], 0)
    df['overs_remaining'] = np.maximum(df['total_overs'] - df['overs_bowled'], 0)
    
    # Calculate required run rate
    df['required_run_rate'] = df.apply(
        lambda row: row['runs_needed'] / np.maximum(row['overs_remaining'], 0.1) 
                    if row['overs_remaining'] > 0 else float('inf'),
        axis=1
    )
    
    # Calculate wicket statistics
    df['wickets_fallen'] = df['is_wicket'].cumsum()
    df['wickets_in_hand'] = 10 - df['wickets_fallen']
    
    # Clean up unnecessary columns
    df = df.drop(columns=['Summary', 'balls_in_current_over', 'Description', 'bowler_good'])
    
    return df

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import numpy as np
import random

import urllib3, socket
from urllib3.connection import HTTPConnection
    
def extraction2(url, target, id, city, stadium, player_names):
    match_id = id
    HTTPConnection.default_socket_options = ( 
            HTTPConnection.default_socket_options + [
            (socket.SOL_SOCKET, socket.SO_SNDBUF, 1000000), #1MB in byte
            (socket.SOL_SOCKET, socket.SO_RCVBUF, 1000000)
        ])
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--no-sandbox")
    #chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-dev-shm-usage")
    
    driver = webdriver.Chrome(options=chrome_options)
    driver.set_page_load_timeout(10)  # Increased timeout
    driver.set_script_timeout(10)     # Increased timeout
    
    try:
        
        try:
            driver.get(url)
            WebDriverWait(driver, 5).until(  # Increased wait time
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.ds-text-tight-l.ds-flex"))
            )
        except Exception as e:
            driver.execute_script("window.stop();")
        
        scroll_pause_time = 1
        
        while True:
            driver.execute_script("window.scrollBy({ top: 200, behavior: 'smooth' });")
            time.sleep(0.2) 
                    
            # Check for the presence of "0.1" in ball numbers
            ball_spans = driver.find_elements(By.CSS_SELECTOR, "span.ds-text-tight-s.ds-font-regular.ds-mb-1.lg\\:ds-mb-0.lg\\:ds-mr-3.ds-block.ds-text-center.ds-text-typo-mid1")
            ball_texts = [span.text.strip() for span in ball_spans if span.text.strip()]
            if "0.1" in ball_texts:
                break
        
        
        time.sleep(1)
        page_source = driver.page_source
        
    except Exception as e:
        print(f"An error occurred: {e}")
        try:
            driver.quit()
        except:
            pass
        return None
    finally:
        try:
            driver.quit()
        except:
            print("Error closing browser")

    
    df = extract_commentary_data(page_source)
    df.to_csv("cricket_commentary_raw.csv", index=False)
    
    try:
        df = preprocess_raw_data(df)
        df.to_csv("commentary_processed.csv", index=False)
        
        df = analyze_match_data(df, int(id), int(target), city, stadium, player_names)
        
        # Save the final processed commentary CSV
        df.to_csv("processed_cricket_commentary.csv", index=False)
        print("Final processed dataset saved as 'processed_cricket_commentary.csv'.")
        
        # Save the enhanced dataset with metadata calculations
        filename = f"{match_id}b.csv"
        df.to_csv(filename, index=False)
        print(f"Dataset saved as '{filename}'.")
        
        
    except Exception as e:
        print(f"Error processing cricket commentary: {e}")
        import traceback
        traceback.print_exc()

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import numpy as np
import random

import urllib3, socket
from urllib3.connection import HTTPConnection

def extraction1(url, target, id, city, stadium, player_names):

    match_id = id
    HTTPConnection.default_socket_options = ( 
            HTTPConnection.default_socket_options + [
            (socket.SOL_SOCKET, socket.SO_SNDBUF, 1000000), #1MB in byte
            (socket.SOL_SOCKET, socket.SO_RCVBUF, 1000000)
        ])
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--no-sandbox")
    #chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-dev-shm-usage")
    
    driver = webdriver.Chrome(options=chrome_options)
    driver.set_page_load_timeout(10)  # Increased timeout
    driver.set_script_timeout(10)     # Increased timeout
    try:
        
        try:
            driver.get(url)
            WebDriverWait(driver, 5).until(  # Increased wait time
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.ds-text-tight-l.ds-flex"))
            )
        except Exception as e:
            driver.execute_script("window.stop();")
        # Add click functionality for innings selection
        try:
            innings_dropdown_trigger = WebDriverWait(driver, 20).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, "ds-popper-wrapper"))
            )
            dropdown = innings_dropdown_trigger[1]
            dropdown.click()
            
            # Locate the innings options within the popup
            innings_options = WebDriverWait(driver, 20).until(
                EC.presence_of_all_elements_located((By.XPATH, "//div[@data-tippy-root]//li[@class='ds-w-full ds-flex']"))
            )
            # Click the first innings tab
            if len(innings_options) >= 1:
                first_innings_tab = innings_options[0]  # First option (index 0)
                tab_title = first_innings_tab.get_attribute("title")
                try:
                    first_innings_tab.click()
                except Exception as e:
                    print(f"Failed to click tab: {e}")
                    driver.execute_script("arguments[0].click();", first_innings_tab)  # Fallback
                    print("Clicked via JavaScript fallback.")
            else:
                print("No innings options found in popup.")
            
            WebDriverWait(driver, 60).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.ds-text-tight-l.ds-flex"))
            )
        except Exception as e:
            print(f"Error selecting innings: {e}")
            # Continue with the script even if innings selection fails
    
        scroll_pause_time = 1
        time.sleep(2)
        from selenium.common.exceptions import StaleElementReferenceException

        while True:
            driver.execute_script("window.scrollBy({ top: 200, behavior: 'smooth' });")
            time.sleep(0.2)
            try:
                ball_spans = driver.find_elements(By.CSS_SELECTOR, 
                    "span.ds-text-tight-s.ds-font-regular.ds-mb-1.lg\\:ds-mb-0.lg\\:ds-mr-3.ds-block.ds-text-center.ds-text-typo-mid1")
                ball_texts = [span.text.strip() for span in ball_spans if span.text.strip()]
            except StaleElementReferenceException:
                # If a stale element error occurs, skip this iteration so that elements are re-fetched in the next loop
                continue
        
            if "0.1" in ball_texts:
                break
        
        
        time.sleep(1)
        page_source = driver.page_source
        
    except Exception as e:
        print(f"An error occurred: {e}")
        try:
            driver.quit()
        except:
            pass
        return None
    finally:
        try:
            driver.quit()
        except:
            print("Error closing browser")
    
    df = extract_commentary_data(page_source)
    df.to_csv("cricket_commentary_raw.csv", index=False)
    
    try:
        df = preprocess_raw_data(df)
        df.to_csv("commentary_processed.csv", index=False)
        
        df = analyze_match_data(df, int(id), int(target), city, stadium, player_names)
        
        # Save the final processed commentary CSV
        df.to_csv("processed_cricket_commentary.csv", index=False)
        
        # Save the enhanced dataset with metadata calculations
        filename = f"{match_id}a.csv"
        df.to_csv(filename, index=False)
        print(f"Dataset saved as '{filename}'.")
        
        
    except Exception as e:
        print(f"Error processing cricket commentary: {e}")
        import traceback
        traceback.print_exc()

In [7]:
import time
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def pre_start(url):
    driver = webdriver.Chrome()
    driver.set_page_load_timeout(10)  # Increased timeout
    driver.set_script_timeout(10)     # Increased timeout
    try:
        try: 
            driver.get(url)
        # Ensure page is fully loaded before interacting
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )
        except Exception as e:
            driver.execute_script("window.stop();")
        
        # Implement a more robust way to scroll
        # Scroll multiple times with pauses to ensure all content loads
        scroll_pause_time = 2
        screen_height = driver.execute_script("return window.screen.height;")
        i = 1
        
        while True:
            # Scroll one screen at a time
            driver.execute_script(f"window.scrollTo(0, {screen_height * i});")
            i += 1
            time.sleep(scroll_pause_time)
            
            # Check if we've reached the bottom
            scroll_height = driver.execute_script("return document.body.scrollHeight;")
            if (screen_height * i) > scroll_height:
                break
                
        # Wait additional time for all dynamic content to load
        time.sleep(1)
        
        # Re-fetch page source after scrolling
        html_content = driver.page_source
        soup = BeautifulSoup(html_content, 'html.parser')
        
        # Fetch stadium info
        stadium_info = soup.find("a", href=re.compile("/cricket-grounds/"))
        if stadium_info:
            title_parts = stadium_info["title"].split(", ")
            if len(title_parts) >= 2:
                stadium_name, city = title_parts[0], title_parts[1]
            else:
                stadium_name, city = title_parts[0], "NA"
        else:
            print("Stadium information not found.")
            stadium_name, city = "NA", "NA"
            
        # Re-fetch tables after waiting
        tables = soup.find_all('table')
        if len(tables) > 4:
            tables = tables[:4]
            
        # Set for player names
        player_names = set()
        
        # Regex pattern to remove (c), †, and any text inside parentheses
        pattern = r"\s*\([^)]*\)|†"
        
        # Extract names from tables
        for table in tables:
            tbody = table.find('tbody')
            if tbody:
                rows = tbody.find_all('tr')
                for row in rows:
                    player_cell = row.find('td')
                    if player_cell:
                        name_span = player_cell.find('span', class_='ds-text-tight-s')
                        if name_span:
                            clean_name = re.sub(pattern, "", name_span.text).strip()
                            if clean_name:
                                player_names.add(clean_name)
        
        return stadium_name, city, player_names
    
    except Exception as e:
        print(f"Error in pre_start: {e}")
        return "NA", "NA", set()
    
    finally:
        driver.quit() 

In [12]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re

def get_target_and_commentary_urls(url, start_match=1):
    driver = webdriver.Chrome()
    extracted_data = []
    match_id = start_match
    driver.set_page_load_timeout(10)  # Increased timeout
    driver.set_script_timeout(10)     # Increased timeout
    try:
        try :
            driver.get(url)
        # Increase wait time for initial page load
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )
        except Exception as e:
            driver.execute_script("window.stop();")
        # Scroll down to load all matches
        scroll_pause_time = 2
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(scroll_pause_time)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        
        # Get fresh page source after all content is loaded
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        matches = soup.find_all('div', class_='ds-p-4 ds-border-y ds-border-line')
        print(f"Found {len(matches)} match containers.")
        driver.quit()
        for idx, match in enumerate(matches[start_match-1:], start=start_match):
            print(f"\nProcessing match {idx}...")
            try:
                link_tag = match.find('a', href=True)
                if not link_tag:
                    print("No link tag found; skipping this match.")
                    match_id += 1
                    continue
                    
                original_url = 'https://www.espncricinfo.com' + link_tag['href']
                
                # Get scorecard URL and fetch preprocessing data
                url_parts1 = original_url.split('/')
                url_parts1[-1] = 'full-scorecard'
                scorecard_url = '/'.join(url_parts1)
                
                print(f"Fetching data from scorecard: {scorecard_url}")
                stadium, city, player_names = pre_start(scorecard_url)
                
                # Get commentary URL
                url_parts = original_url.split('/')
                url_parts[-1] = 'ball-by-ball-commentary'
                commentary_url = '/'.join(url_parts)
                
                # Extract city from the original match data
                local_city = None
                details_div = match.find('div', class_='ds-text-tight-s ds-font-regular ds-truncate ds-text-typo-mid3')
                if details_div:
                    span = details_div.find('span', class_='ds-text-tight-s ds-font-medium ds-text-typo')
                    if span:
                        next_text = span.next_sibling
                        if next_text and isinstance(next_text, str):
                            city_part = next_text.strip().lstrip('•').strip()
                            local_city = city_part.split(',')[0].strip()
                
                # Use the city from scorecard if available, otherwise use the one from match page
                if city == "NA" and local_city:
                    city = local_city
                
                # Extract target
                target = None
                team_scores = match.find_all('div', class_='ci-team-score')
                for score_div in team_scores:
                    score_text_div = score_div.find('div', class_='ds-text-compact-s ds-text-typo ds-text-right ds-whitespace-nowrap')
                    if score_text_div:
                        target_span = score_text_div.find('span', class_='ds-text-compact-xs ds-mr-0.5')
                        if target_span and target_span.text.strip():
                            target_match = re.search(r'T:(\d+)', target_span.text)
                            if target_match:
                                target = int(target_match.group(1))
                                break
                
                if target is None:
                    target = 0
                    print("No target score found for this match.")
                
                # Append all data
                extracted_data.append({
                    'match_id': idx,
                    'city': city,
                    'stadium': stadium,
                    'target_score': target,
                    'commentary_url': commentary_url,
                    'player_names': player_names
                })
                
                # Call extraction functions with fresh data
                print("First innings Started")
                extraction1(commentary_url, target, idx, city, stadium, player_names)
                print("Second innings Started")
                extraction2(commentary_url, target, idx, city, stadium, player_names)
                
                
            except Exception as e:
                print(f"Error processing match {idx}: {e}")
                            
                
            # Wait between match processing to avoid overloading
            time.sleep(3)
        
        return extracted_data
    
    except Exception as e:
        print(f"An error occurred in main process: {e}")
        return extracted_data  
    
    finally:
        print("Closing main browser...")
        print("\nData extraction complete.")


if __name__ == "__main__":
    schedule_url = "https://www.espncricinfo.com/series/pepsi-indian-premier-league-2015-791129/match-schedule-fixtures-and-results"
    print("If the script stopped previously, you can resume from a specific match.")
    user_input = input("Enter the match number to start from (or press Enter to start from 1): ")        
    start_match = int(user_input) if user_input else 1
    
    print(f"Starting scraping from match number {start_match}")
    match_details = get_target_and_commentary_urls(schedule_url, start_match)
    

If the script stopped previously, you can resume from a specific match.


Enter the match number to start from (or press Enter to start from 1):  


Starting scraping from match number 1
Found 60 match containers.

Processing match 1...
Fetching data from scorecard: https://www.espncricinfo.com/series/pepsi-indian-premier-league-2015-791129/kolkata-knight-riders-vs-mumbai-indians-1st-match-829705/full-scorecard
First innings Started
Dataset saved as '1a.csv'.
Second innings Started
Final processed dataset saved as 'processed_cricket_commentary.csv'.
Dataset saved as '1b.csv'.

Processing match 2...
Fetching data from scorecard: https://www.espncricinfo.com/series/pepsi-indian-premier-league-2015-791129/chennai-super-kings-vs-delhi-daredevils-2nd-match-829707/full-scorecard
First innings Started
Dataset saved as '2a.csv'.
Second innings Started
Final processed dataset saved as 'processed_cricket_commentary.csv'.
Dataset saved as '2b.csv'.

Processing match 3...
Fetching data from scorecard: https://www.espncricinfo.com/series/pepsi-indian-premier-league-2015-791129/kings-xi-punjab-vs-rajasthan-royals-3rd-match-829709/full-scorecard
